In [1]:
from langchain_google_vertexai import VertexAI
llm = VertexAI(model_name="gemini-1.5-pro")
print(llm.invoke("Which question can you answer?"))

Please, **ask me a question!**  I'm happy to try and answer anything you're curious about. I'm good at things like:

* **Providing summaries of topics**
* **Answering factual questions**
* **Generating creative text formats** (like poems, code, scripts, etc.)
* **Translating languages**
* **And much more!**

The more specific your question, the better I can understand and respond to it. 😊 



In [2]:
task = "Write a poem about Google CLoud and LangCHain"
for chunk in llm.stream(task):
    print(chunk)

In
 digital realms where data flows,
Two forces meet, their power grows.

Google Cloud, a vast expanse,
Where code takes flight, in rhythmic dance
.

With servers humming, a symphony,
Of processing might, for you and me.
Storage boundless, a digital sea,
Where terabytes swim
, eternally.

And then there's LangChain, a weaver of words,
With language models, like flocks of birds.
Transforming text,
 with AI's grace,
Unlocking insights, at a rapid pace.

It chains commands, in intricate flow,
From question to answer, knowledge to bestow.
Connecting models, large and grand,
A symphony of language
, at our command.

Together they stand, a powerful pair,
Cloud's infrastructure, LangChain's flair.
Building applications, smart and bold,
Where language reigns, and stories unfold.

So let us explore
, this digital art,
Where Google Cloud and LangChain impart,
A future bright, with endless might,
In the realm of language, shining bright. 



In [3]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

for chunk in llm.stream(
    "Write a poem about Sango Ota and Nigeria",
    temperature=0.9,
    max_output_tokens=100,
    stop=["."],
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH
        : HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    }
):
    print(chunk)

In
 Ogun's embrace, where industry thrives,
Lies Sango Ota, where
 ambition strives
